# HW3 MAT258A Irene Kim 999477003

Exercise 1

-First this is a test script that applies to a simple function(quadratic) that converges at one step.

In [1]:
using Toms566

In [2]:
x = 0
function ff(x) 
    (x-3)^2 + 5
end
function gg(x) 
    2*(x-3)
end
H = 2 #here we have positive constant hessian so don't need to alter

function test(x0, optTol = 1e-6)
    x = x0
    f = ff(x)
    itn = 0
    while abs(gg(x))>optTol*abs(gg(x0))
        d = -gg(x)/2
        x = x+d
        f = [f;ff(x)]
        itn = itn+1
    end
    [itn; x; ff(x)]
end

test(0, 1e-6)

3-element Array{Float64,1}:
 1.0
 3.0
 5.0

Now, we try the simple Newton's method to the 18 problems.

In [3]:
function NWTNsimple(p::Toms566.Problem, optTol, maxItn)
    x = p.x0
    f = p.obj(x)
    g = p.grd(x)
    H = p.hes(x)
    itn = 0
    
    while norm(p.grd(x))> optTol*norm(p.grd(p.x0))
        #first compute d ; the descent direction
        #we use hessian no matter if it is positive definite or not
        d = p.hes(x)\(-p.grd(x))
        x = x + d #newton's method alpha equals 1
        f = [f;p.obj(x)]
        itn = itn+1
        if itn == maxItn
            break
        end
    end
    [itn; p.obj(p.x0); p.obj(x)]
end

for i in 1:18
    p = Problem(i)
    @printf("%3i %3i %12.2e %12.2e\n", i,NWTNsimple(p, 1e-6, 500)...)
end

  1  32     2.50e+03     4.10e-12
  2 500     7.79e-01     6.10e+00
  3   2     3.89e-06     1.13e-08
  4  15     1.14e+00     2.00e+00
  5   7     1.03e+03     4.95e-08
  6  17     9.39e+10     5.08e+03
  7 500     3.00e+01     3.00e+68
  8  11     5.45e+09     4.78e+01
  9  68     2.87e+05     8.81e+01
 10 500     1.00e+12     1.00e+12
 11 500     7.93e+06    3.34e+111
 12 500     1.21e+01     1.35e+65
 13  12     2.01e-03     2.13e-18
 14 500     4.84e+02     1.23e+19
 15 500     3.23e+03     4.16e+44
 16   1     1.42e+01     1.42e+01
 17  11     1.92e+04     7.88e+00
 18   8     1.39e-02          NaN


In [4]:
#Modified Newton's(modifying the Hessian matrix) method with backtracking
function NWTNmod2bt(p::Toms566.Problem, optTol, eps, maxItn)
    x = p.x0
    f = p.obj(x)
    g = p.grd(x)
    itn = 0
    while norm(p.grd(x))>optTol*norm(p.grd(p.x0))
        #first compute d ; the descent direction
        #first we check if the hessian is positive definite
        if minimum(eig(p.hes(x))[1])<=0
            D, V = eig(p.hes(x))
            y = map(x->max(x,eps), D)
            H = V*Diagonal(y)*V'
            #eye(size(p.hes(x))[1], size(p.hes(x))[2])
        else 
            H = p.hes(x)
        end
        #now H is positive definite
        d = H\(-p.grd(x))
        #will do backtrack on alpha
        alpha = 1
        while p.obj(x+alpha*d)>p.obj(x) + alpha*0.4*dot(d,p.grd(x))
                alpha = alpha/2
            end
        x = x + alpha*d
        itn = itn+1
        if itn == maxItn 
            break 
        end
    end
    [itn; p.obj(p.x0); p.obj(x)]
end

for i in 1:18
    p = Problem(i)
    @printf("%3i %3i %12.2e %12.2e\n", i, NWTNmod2bt(p, 1e-6, 1, 100)...)
end

  1   8     2.50e+03     3.96e-14
  2 100     7.79e-01     2.24e-01
  3   2     3.89e-06     1.13e-08
  4  91     1.14e+00     2.07e-14
  5  13     1.03e+03     8.20e-09
  6   7     9.39e+10     1.17e+03
  7 100     3.00e+01     3.27e-02
  8  11     5.45e+09     4.78e+01
  9  18     2.87e+05     8.81e+01
 10   7     1.00e+12     3.16e-30
 11 100     7.93e+06     1.43e+06
 12 100     1.21e+01     3.88e+00
 13 100     2.01e-03     6.49e-06
 14 100     4.84e+02     1.81e+02
 15 100     3.23e+03     5.84e+01
 16   9     1.42e+01     1.06e-17
 17   8     1.92e+04     7.88e+00
 18 100     1.39e-02     7.60e-03


Also we get # 2, 7, 11-15, 18 not converging in 100 steps. Now we try a different alteration of the hessian matrix to try the remaining problems, by altering the eigenvalues to be maximum of 1 and absolute value of the eigenvalue.

In [8]:
function NWTNmod1bt(p::Toms566.Problem, optTol, eps, mu, maxItn)
    x = p.x0
    f = p.obj(x)
    g = p.grd(x)
    itn = 0
    while norm(p.grd(x))>optTol*norm(p.grd(p.x0))
        #first compute d ; the descent direction
        #first we check if the hessian is positive definite
        if minimum(eig(p.hes(x))[1])<=0
            D, V = eig(p.hes(x))
            y = map(x->max(abs(x),eps), D)
            H = V*Diagonal(y)*V'
            #eye(size(p.hes(x))[1], size(p.hes(x))[2])
        else 
            H = p.hes(x)
        end
        #now H is positive definite
        d = H\(-p.grd(x))
        #will do backtrack on alpha
        alpha = 1
        while p.obj(x+alpha*d)>p.obj(x) + alpha*mu*dot(d,p.grd(x))
                alpha = alpha/2
            end
        x = x + alpha*d
        itn = itn+1
        if itn == maxItn 
            break 
        end
    end
    [itn; p.obj(p.x0); p.obj(x)]
end

NWTNmod1bt (generic function with 1 method)

In [10]:
for i in [2;7;11:15;18]
    p = Problem(i)
    @printf("%3i %3i %12.2e %12.2e\n", i, NWTNmod1bt(p, 1e-6, 1, 1e-4, 100)...)
end

  2 100     7.79e-01     2.62e-01
  7 100     3.00e+01     2.91e-02
 11  18     7.93e+06     8.58e+04
 12 100     1.21e+01     3.88e+00
 13 100     2.01e-03     6.47e-06
 14 100     4.84e+02     8.17e+01
 15 100     3.23e+03     3.76e-01
 18 100     1.39e-02     7.45e-03


We see that # 11 converged. Hence now the remaining problems are # 2, 7, 12, 13, 14, 15, 18. Now, we'll try the quasi-Newton BFGS method.

In [14]:
#Quasi Newton method
function NWTNquasi(p::Toms566.Problem, optTol, maxItn)
    itn = 0
    x = p.x0
    D, V = eig(p.hes(p.x0))
    y = map(x->max(x,1), D)
    H = V*Diagonal(1./y)*V'#inversion of modified hessian matrix
    while norm(p.grd(x))>optTol*norm(p.grd(p.x0))
        #first compute d ; the descent direction
        #first we check if the hessian is positive definite
        d = H*(-p.grd(x))
        
    #finding alpha satifying Wolfe’s condition
        alpha = 1
        while p.obj(x+alpha*d)>p.obj(x) + alpha*1e-3*dot(d,p.grd(x))
            alpha = alpha*(1/2)
            if alpha <= 1e-6
                break
            end
        end
        while dot(d,p.grd(x+alpha*d))<dot(0.9*d,p.grd(x))
            alpha = alpha*(1/2)
            if alpha <= 1e-6
                break
            end
        end

        x = x+alpha*d
        s = alpha*d
        y = p.grd(x)-p.grd(x-s)
        ro = 1/dot(y,s)
        H = (eye(H)-ro*s*y')*H*(eye(H)-ro*y*s') + ro*s*s'
        itn = itn + 1
        if itn == maxItn 
            break 
        end
    end
    [itn; p.obj(p.x0); p.obj(x)]
end

for i in 1:18
    p = Problem(i)
    @printf("%3i %3i %12.2e %12.2e\n", i,NWTNquasi(p, 1e-8, 1000)...)
end

  1  27     2.50e+03     1.30e-16
  2 111     7.79e-01     8.04e-16
  3   4     3.89e-06     1.13e-08
  4 157     1.14e+00     2.97e-18
  5 1000     1.03e+03     4.62e-03
  6 523     9.39e+10     1.57e+05
  7  95     3.00e+01     1.40e-06
  8  51     5.45e+09     4.90e-02
  9 1000     2.87e+05     8.81e+01
 10  38     1.00e+12     9.09e-19
 11  21     7.93e+06     8.58e+04
 12  33     1.21e+01     2.34e-15
 13  64     2.01e-03     3.95e-06
 14 234     4.84e+02     4.27e-11
 15 262     3.23e+03     1.31e-08
 16  13     1.42e+01     8.34e-17
 17 1000     1.92e+04     7.87e+00
 18 336     1.39e-02     5.39e-03


We see in the result that the problems # 2, 7, 12-15, 18 all now have converged with alteration of the optTol value(smaller) and more steps.  

###Exercise 2

In [15]:
Admissions = readdlm("/Users/Irene/Documents/UCD/2015fall/MAT258a/hw2/binary.csv", ',', header=true);

y = Admissions[1][1:400,1];#response variable 400x1
u0 = Admissions[1][1:400,2:3]*float(Diagonal([1/400, 1]));#GRE and GPA
u = (cat(2,[1 for i = 1:400],u0))';#the vector(data) that we do inner product to the parameters beta and a 400x3

#define the objective, gradient and hessian functions
#objective function
function f(x)#x is the parameter values and f is the negative log likelihood 
    obj = 0
    for i = 1:400
        obj = obj - y[i,]*(x'*u[:,i]) + log(1+exp(x'*u[:,i]))
    end
    obj
end

function gf(x)#compute the graident
    g = [0;0;0]
    for i = 1:400
        g = g - vec(y[i,].*u[:,i] - ( (exp(x'*u[:,i])) / (1+exp(x'*u[:,i])) ).*u[:,i])  
    end
    g
end

function hf(x)#compute the hessian
    h = 0
    for i = 1:400
        h = h + (exp(x'*u[:,i]))/(1 + exp(x'*u[:,i])).*u[:,i]*u[:,i]'
    end
    h
end

hf (generic function with 1 method)

In [16]:
#modified newton's method(hessian alteration) with backtracking alpha
function NWTN(x0, optTol, eps, mu, maxItn)
    x = x0
    itn = 0
    for itn = 1:maxItn
        
        if norm(gf(x))<=optTol*norm(gf(x0))
            break
        end
        
        #to decide the step direction, first alter hessian as needed
        D, V = eig(hf(x))

        if minimum(D)<=0
            pD = map(x->max(x,eps), D)
            h = V*pD*V'
        else
            h = hf(x)
        end

        d = -h\gf(x)#we found the descending direction!
        #now we need to choose alpha
        alpha = 1
        while ( f(x+alpha*d) - f(x) + alpha*mu*(d'*gf(x)) )[]>0
            alpha = (1/2)*alpha
            if alpha<1e-7
                break
            end
        end
        x = x+alpha*d
        itn = itn + 1 
    end
    [itn;x;f(x)]
end

NWTN (generic function with 1 method)

In [17]:
x0 = [0;0;0]
NWTN(x0, 1e-6, 1, 0.2, 1000)

5-element Array{Float64,1}:
  16.0     
  -4.94938 
   1.07627 
   0.754688
 240.172   

Here we see that there were 16 iterations to convergence where in the last homework with 1st order method there were 20+ iterations needed for convergence. We see that second order method performs better. 